In [1]:
from openpyxl import load_workbook
import pandas as pd
import os
import GladUtil as glad

''' 본사시상금을 추출하는 스크립트
    입출력 파일명과 업적월,지급월을 수정한 후에 실행하면 출력파일에 시트가 추가됨
'''

infile ='E:/글로벌백업(180329)/04.수수료작업/202112시상/202112 KJ시상정산-배포.xlsx'
outfile = 'E:/글로벌백업(180329)/04.수수료작업/시상이자/본사시상정산.xlsx'

sheet_nm = ['생보스팟','한화','KB','농협','MG','롯데','DB','현대','메리츠','흥국','AIG','삼성','처브','하나']
업적월 = 202112
지급월 = 202201

FIX_VAL = [업적월,지급월,'']

wb = load_workbook(infile,data_only=True)
ws = wb['한화']
df = pd.DataFrame(columns=['업적월','지급월','회사','지점','사원','시상금','비고'])


In [2]:
''' 각 시트에서 본사 시상금만 추출 ''' 
for sh in sheet_nm:
    ws = wb[sh]
    FIX_VAL[2] = sh
    
    st_yn = 'n'
    for cell in ws['a']:
        if cell.value:
            if '본사' in cell.value:  # 본사 시상금은 시트의 맨 아래에 있어야 함
                st_yn = 'y'
            if  st_yn =='y':
                if '본사' in cell.value or '글래드' in cell.value:  #본사시상명 또는 지점있는 row
                    r = cell.row
                    vals = []
                    if '본사' in cell.value:
                       titl = cell.value
                       vals = [ cell.value, ' ',' ',' ']
                    if '글래드' in cell.value:
                        vals = [ x.value for x in ws[r] if x.value !=None ]
                    val = [ vals[0],vals[1],vals[len(vals)-1] ]  #지점,이름,시상금
                    val.append(titl)
                    df.loc[len(df)] = FIX_VAL + val

In [21]:
##--공동정산 대상건만 추출
df2 = glad.Share_Proc(df,"공동","본사x")  # 본사 헤더는 추출하지 않음
df2['시상금'] = df2['시상금'].astype('int32',errors='ignore')  
df2 = df2[df2['시상금']!=0]

w_mode = 'a' if os.path.isfile(outfile) else 'w'  #파일있으면 append mode
with pd.ExcelWriter(outfile, mode = w_mode, engine="openpyxl") as writer:
     df2.to_excel(writer, sheet_name=str(업적월), index=False)
        
print("완료")

완료
